# Model Interpretability: SHAP

In [ ]:
# Obviously
import pandas as pd
import numpy as np

# Some sklearn tools for preprocessing.
# ColumnTransformer was introduced in 0.20 so make sure you have this version
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Our algorithms, by from the easiest to the hardest to intepret.
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm.sklearn import LGBMClassifier

### The Dataset

In [ ]:
# Load data and create lists of numerical and categorical features

df = pd.read_csv("data/bank.csv")

# Get X, y
y = df["y"].map({"no": 0, "yes": 1})
X = df.drop("y", axis=1)

num_features = ["age", "campaign", "pdays", "previous"]

cat_features = ["job", "marital", "education","default", "housing", "loan", "contact", 
                "month", "day_of_week", "poutcome"]

Description of features:
1. age (numeric)
2. job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3. marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4. education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5. default: has credit in default? (categorical: 'no','yes','unknown')
6. housing: has housing loan? (categorical: 'no','yes','unknown')
7. loan: has personal loan? (categorical: 'no','yes','unknown')
8. contact: contact communication type (categorical: 'cellular','telephone') 
9. month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10. day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11. campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
12. pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
13. previous: number of contacts performed before this campaign and for this client (numeric)
14. poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')

In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.3, random_state=42)

Let's create a preprocessor as before, and use the same gymnastics to get nice name for our new dummy features. 
Here we'll keep both the raw data `X_train` and processed one `X_train_processed` as we will need both of them for LIME.

In [ ]:
preprocessor = ColumnTransformer([("numerical", "passthrough", num_features), 
                                  ("categorical", OneHotEncoder(sparse=False, handle_unknown="ignore"),
                                   cat_features)])

In [ ]:
# Train preprocessor
preprocessor.fit(X_train)

# Get the list of categories generated by the process
ohe_categories = preprocessor.named_transformers_["categorical"].categories_

# Create nice names for our one hot encoded features
new_ohe_features = [f"{col}__{val}" for col, vals in zip(cat_features, ohe_categories) for val in vals]

# Create a new list with all names of features
all_features = num_features + new_ohe_features

# Save processed data
X_train_processed = pd.DataFrame(preprocessor.transform(X_train), columns=all_features)
X_test_processed = pd.DataFrame(preprocessor.transform(X_test), columns=all_features)

### The models

Since we've tuned and trained the models in the previous notebook, here we'll just do everything at once so we can focus on interpreting them.

In [ ]:
# Logistic Regression
lr_model = LogisticRegression(C=1.3, class_weight="balanced", solver="liblinear", random_state=42)
lr_model.fit(X_train_processed, y_train)

# Decision Tree
dt_model = DecisionTreeClassifier(class_weight="balanced", max_depth=10, min_samples_split=.01)
dt_model.fit(X_train_processed, y_train)

# Random Forest
rf_model = RandomForestClassifier(class_weight="balanced", n_estimators=150, n_jobs=-1, 
                                                      min_samples_split=.01, max_depth=15)
rf_model.fit(X_train_processed, y_train)

# LightGBM
lgb_model = LGBMClassifier(class_weight="balanced", n_jobs=-1, 
                                              max_depth=10, min_child_samples=40, n_estimators=50)
lgb_model.fit(X_train_processed, y_train)

## SHAP to interpret local predictions

In [ ]:
import shap
# Need to load JS vis in the notebook
shap.initjs() 

SHAP has a generic explainer that works for any model, called `KernelExplainer`. This explainer requires an arbitrary model and some training data in order to calibrate itself. This fully agnostic method is slow and only computes an approximation of the SHAP values, so `shap` also provides special explainers that are optimised for specific classes of models. The `TreeExplainer` is optimised for tree based models, and is what we will use here with our LGB model (the hardest to intepret). 

`TreeExplainer` also has the advantage that it can "calibrate" itself using properties of trees directly, instead of estimating values from an external dataset, that means that we only need to provide a model and `shap` does the rest.

To get started, you need to create a SHAP explainer object from your model. 

Create a new explainer using `shap.TreeExplainer`, here you can directly pass your model stored inside the `lgb_model` pipeline:

Now that we have an explainer object, we can start using it to compute SHAP values on our data. Here we're interested in explaining specific predictions, let's choose a row in our test set first:

In [ ]:
i = 4

observation = X_test_processed.iloc[[i]]
print(f"Observation true label: {y_test.iloc[i]}")
print(f"Observation predicted label (proba): {lgb_model.predict_proba(observation)[:,1]}")

observation

The observation we selected corresponds to a 27 years old prospect that ended up subscribing to our plan, and our model predicted that outcome with a probability of .89, not bad! 

The first step for interpretation is to compute SHAP values for this observation. SHAP values are a way to know the contribution of each specific attribute of the observation in the final prediction: for instance, how much of an impact the fact that the prospect is 27 had on our prediction?  What about the fact that they were called on a Wednesday? etc...

To compute SHAP value, you can use the method `shap_values` from your model explainer and pass your observation. Do so and call the output `shap_values`:

Check the dimension of the returned matrix:

For binary classification, `shap_values` returns a single numpy array with the SHAP value associated to each of the features. For our given observation, SHAP value per feature can be positive or negative, and if we sum them we get the difference between the value our model predicted and its "expected value".

If you use the model agnostic `KernelExplainer`, those values will be expressed as probabilities. With the `TreeExplainer` though, the unit will be specific to the library/model you are using. For lightbm's classifier that is `log_odds`. let's do some sanity check!

In [ ]:
# If you need to check in what unit the output is, run the following:
explainer.model.tree_output

Let's start with the `expected_value`: it corresponds to the average output of our model, here it will be expressed as a log_odds of the predictions of our models on the training set:

In [ ]:
def average_log_odds(predictions):
    return np.mean(np.log(predictions / (1 - predictions)))

With `lgb_model`, generate predictions from the training data and compute the average log odds using the function above (don't forget to get the prediction for class 1 only -- predict_proba will return two values per row)

Check that it is equal to your explainer's `expected_value`:

Great, now we can compute the difference between our model's prediction for this specific observation (it needs to be expressed as a log_odd) and the expected value: this difference is what `shap` is aiming to explain through SHAP values:

In [ ]:
# Compute the difference between your model's prediction on the observation and the expected value


Verify that this matches the sum of your SHAP values:

`shap` implements a `force_plot` function that allows to nicely plot the shap values for our observation, call `force_plot` passing:
- your model's `expected_value`
- your observation's `shap_values`
- your observation (the row)

By default this will plot shap_values as log_odds, but you can display them as probabilities by passing `link="logit"`

This plot makes it easier to see the impact of each feature. Here the base value (equivalent of the expected value) is at 0.39 (most people did not subscribe to a plan so this is the most likely outcome) whilst our model predicted 0.90 chance of the person subscribing.

In pink we see the features that justifies this shift from base value to output value: the fact that the prospect was contacted 3 days after the previous campaign seems to have the highest impact (a contribution of about 0.20), the month being June also seems to have a positive impact. In blue, only one feature is impacting the prediction in the other direction, the fact that this person was contacted 4 times.

With this tool at hand, we can now go back to the marketing team and explain what our model seems to look at here to make predictions, it would be interesting to compare with what they think might have happened. A few questions that we might have:
- Did something special happened in June -- is our model consistently picking June as an explanation for successful campaigns?
- Why is pdays=3 important? Would it be better with a bit more days? A bit less?
- What's wrong with having contacted the person 4 times? 
- Can pdays and campaign be linked? Or maybe linked to other features in the data (different demographics might prefer more or less pushy campaigns)?

The marketing team can probably help with some of those questions, but in the meantime, we can investigate further by investigating our model at a global scale instead of local. `shap` provides efficient techniques to do so!

## SHAP for global interpretation of a model

If we compute SHAP values for multiple observations at a time, `shap` is able to plot them with `force_plot` in a similar way as above, but for all at once, let's take a look. First select 1000 observations at random from the test set, call the resulting dataframe `observations`:

Use our explainer (same as before, calibrated on our `lgb_model`) to compute SHAP values for our observations:

Use `force_plot` again, but this time passing all shapley values and all observations:

We can see our 1000 samples on the x-axis. The y-axis corresponds to the same scale we were looking at before, where blue values corresponds to the probability decreasing, red increasing. 

By default our observations our sorted by similarity, we can see on the left a group of samples which have a high probability of subscribing to the plan and for which `pdays` have a high impact (large pink area -- you can hover with your mouse to see the details). In the center we have a majority of samples around 50% chance of either subscribing or not and with feature such as the month, cellular and default being important. Then on the right we can see samples that have a really low probability of subscribing to our plan.

You can use the dropdown at the top to sort the samples by a specific feature, try to sort them by `pdays`, what do you observe? If it is hard to see with all shap values displayed, you can filter to keep only pdays related shap values using the dropdown on the left.

It looks like smaller values of pdays have a large positive impact on the probability of subscribing, whilst larger values seem less relevant or even have a negative impact.

Can you do the same with `campaign` and `age`, what do you observe?

Using `force_plot` on multiple SHAP values is really powerful, but still requires to go through features one by one to understand the model. `shap` provides another plot, the `summary_plot` that can be seen as an overall feature importance plot with extra information about the way each feature impacts outcomes.

Use `summary_plot` passing your subset of SHAP values and observations:

Here each point corresponds to a SHAP value for a given feature for a given samples. Features are grouped on the y-axis and the x-axis represent the SHAP value, either negative or positive. The colour correspond to the value of the feature, either high or low.

For example, `contact_cellular` will be red if 1 and blue if 0. Here it looks like for people that have been contacted by phone (contact_cellular==1==red), this has contributed positively to their chance of subscribing, whilst not having a phone contributed negatively (blue dots on the left side). The impact is fairly low on both sides though, between -1 and 1.

`pdays` is interesting too, here high values (red) of pdays seem to have a small negative impact on the predictions, but a low value, a couple of days has a really high positive impact (that's what we've observed multiple times already).

`campaign` has an opposite behaviour: a really large negative impact for high values (when prospects are called too many times).

We can find interesting information about months as well, it looks like October and March have quite high impact for some predictions in red (value==1), whilst month like July don't seem to matter much. 

Finally, `shap` also implements a way to plot SHAP values as interaction between two features. 

Call `dependence_plot` on the feature `age`. You will need to pass your shap_values and observations.

We can see again that age seems to have a high impact for the youngest and oldest prospects but not as much in between. `dependence_plot` automatically picked a feature that was correlated with `age` to be the dependent one here, but this can be changed by setting  `interaction_index` to any other features you want.

Try multiple combinations of features, and try to make sense of what you observe.

`shap` is really powerful and allows to understand your model both locally and globally. 

A word of warning though, it can get extremely slow to compute SHAP values using the model-agnostic KernelExplainer. That means that if your model is tree based (tree explainer), neural networks (deep explainer) or linear, you'll be able to get the best out of `shap` with good performance. But for other models, such as KNN, SVM or even a custom sklearn Pipeline object allowing you to work with categorical features directly instead of dummies, you will have to either use the slow `KernelExplainer` or prefer other more approximative techniques such as LIME for local explanations or permutation importance for global ones.